In [1]:
import pandas as pd
import numpy as np
import math

#### Read Data

In [3]:
df = pd.read_csv('D:\FirstSemMT\ML\Assignments\lung.tab',sep='\t')
df.shape

(205, 12601)

In [4]:
df.head()

,class,38691_s_at,37864_s_at,33273_f_at,33274_f_at,33501_r_at,33500_i_at,33499_s_at,41164_at,38194_s_at,...,41848_f_at,32086_at,33886_at,31781_at,AFFX-BioC-3_at,41422_at,39964_at,36120_at,40571_at,36312_at
0,discrete,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,...,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous,continuous
1,class,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AD,63.2,4196.25,3306.35,3330.86,1609.47,1597.32,1233.89,255.14,3036.53,...,-17.79,18.63,51.04,-13.74,-29.12,-28.31,20.25,28.35,4.06,-19.41
3,AD,965.47,6207.61,7077.04,6968.59,6569.86,6419.19,6908.34,4785.76,4562.19,...,-5.74,5.94,28.23,-4.68,-13.18,-13.18,21.86,9.12,11.24,8.06
4,AD,2940.51,6858.12,6927.79,6495.99,5273.47,4672.48,5474.67,2140.99,5120.39,...,-17.225,4.725,17.28,-6.59,-17.97,-16.07,10.195,17.285,6.92,-11.09


In [5]:
df.drop(df.index[df[df.columns[0]] == 'discrete'], inplace = True)
df.drop(df.index[df[df.columns[0]] == 'class'], inplace = True)
label = df[df.columns[0]] 
df.drop([df.columns[0]], axis = 1, inplace = True)
df = df.values
samples,features = df.shape
print(samples,features)

203 12600


In [6]:
data = df.astype(np.float)
data.shape

(203, 12600)

#### Import train_test_split function and Split dataset into training set and test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2,random_state=100) 

#### Find Mutual Info for every feature 

In [8]:
ig = np.zeros(features)

c1 = np.where(y_train=='AD ')[0]
c2 = np.where(y_train=='NL ')[0]
c3 = np.where(y_train=='SMCL ')[0]
c4 = np.where(y_train=='SQ ')[0]
c5 = np.where(y_train=='COID ')[0]

u = np.array([c1.shape[0],c2.shape[0],c3.shape[0],c4.shape[0],c5.shape[0]])
ans1=0
ic = 0
for p in range(5):
    ans1 = ans1 - ((u[p]/u.sum())*math.log2(u[p]/(u.sum())))
ic = ans1    

for i in range(features): 
    minElement = np.amin(X_train[:,i])
    maxElement = np.amax(X_train[:,i])
    diff = (maxElement-minElement)/2
    
    a = minElement
    b = a+diff
    c = a+(2*diff)
    
    x = np.where(np.logical_and(X_train[:,i]>=a,X_train[:,i]<b))[0] 
    y = np.where(np.logical_and(X_train[:,i]>=b,X_train[:,i]<c))[0] 
    
    xc1=np.intersect1d(x,c1)
    yc1=np.intersect1d(y,c1)
    xc2=np.intersect1d(x,c2)
    yc2=np.intersect1d(y,c2)
    xc3=np.intersect1d(x,c3)
    yc3=np.intersect1d(y,c3)
    xc4=np.intersect1d(x,c4)
    yc4=np.intersect1d(y,c4)
    xc5=np.intersect1d(x,c5)
    yc5=np.intersect1d(y,c5)
    
    v = np.array([[xc1.shape[0],yc1.shape[0]],[xc2.shape[0],yc2.shape[0]],[xc3.shape[0],yc3.shape[0]],[xc4.shape[0],yc4.shape[0]],[xc5.shape[0],yc5.shape[0]]])
    ans2=0
    for p in range(2):
        for q in range(5):
            if(v[q][p]!=0):
                ans2 = ans2 - ((v[q][p]/v.sum())*math.log2(v[q][p]/(v[0][p]+v[1][p]+v[2][p]+v[3][p]+v[4][p])))
    ig[i] = ic-ans2
print(ig)
    

[0.24079281 0.1809088  0.14966442 ... 0.07827839 0.01959299 0.07901625]


#### Sort indexes in descending order of mutual info

In [9]:
sort_idxs1=(-ig).argsort()[:ig.shape[0]]
sort_idxs1

array([2028,  370, 4018, ..., 1803, 7454,  300], dtype=int64)

#### Select top features (less than 20%) from mutual info

In [10]:
filt_idxs1 = sort_idxs1[0:int((data.shape[1])*(0.2))]

In [11]:
X_train = X_train[:,filt_idxs1]

#### Find f value for every feature

In [12]:
total_class = 5

#degree of freedom
dof_between = total_class-1  #dof_between = 4
dof_within = data.shape[0]-total_class   #dof_within = 198
dof_total = dof_between+dof_within

Dof_critical = 3.92  #at (4,198) for alpha=0.05 

In [13]:
c1 = np.where(y_train=='AD ')[0]
c2 = np.where(y_train=='NL ')[0]
c3 = np.where(y_train=='SMCL ')[0]
c4 = np.where(y_train=='SQ ')[0]
c5 = np.where(y_train=='COID ')[0]

#to calculate no. of samples in both the class
c_dash1 = np.where(label=='AD ')[0]
c_dash2 = np.where(label=='NL ')[0]
c_dash3 = np.where(y_train=='SMCL ')[0]
c_dash4 = np.where(y_train=='SQ ')[0]
c_dash5 = np.where(y_train=='COID ')[0]

#mean of each class
m1 = X_train[c1,:].mean(axis=0)
m2 = X_train[c2,:].mean(axis=0)
m3 = X_train[c3,:].mean(axis=0)
m4 = X_train[c4,:].mean(axis=0)
m5 = X_train[c5,:].mean(axis=0)
m_grand = X_train[:,:].mean(axis=0)

#variance of each class
v1 = X_train[c1,:].var(axis=0)
v2 = X_train[c2,:].var(axis=0)
v3 = X_train[c3,:].var(axis=0)
v4 = X_train[c4,:].var(axis=0)
v5 = X_train[c5,:].var(axis=0)

ss_total = np.sum((X_train[:,:] - m_grand)**2, axis=0)
#print(ss_total)

ss_within = (c_dash1.shape[0]-1)*v1 + (c_dash2.shape[0]-1)*v2 +(c_dash3.shape[0]-1)*v3 +(c_dash4.shape[0]-1)*v4 +(c_dash5.shape[0]-1)*v5
ss_between = ((c_dash1.shape[0])*(m1-m_grand)**2) +((c_dash2.shape[0])*(m2-m_grand)**2) +((c_dash3.shape[0])*(m3-m_grand)**2)+((c_dash4.shape[0])*(m4-m_grand)**2)+((c_dash5.shape[0])*(m5-m_grand)**2)

ms_between = ss_between/dof_between
ms_within = ss_within/dof_within

#f-stat
f = ms_between/ms_within
print(f)
print(f[f>Dof_critical].shape)

[ 79.92567402 110.16455201 117.24078491 ...  13.64418942  21.80001133
  14.88402969]
(2519,)


#### Sort indexes in descending order of f value

In [14]:
sort_idxs2=(-f).argsort()[:f.shape[0]]
sort_idxs2

array([  48,   18, 1583, ..., 2483, 2200, 2025], dtype=int64)

#### Select 66% features from f value out of selected features from mutual info

In [15]:
filt_idxs2 = sort_idxs2[0:int((X_train.shape[1])*(0.67))]

In [16]:
X_train = X_train[:,filt_idxs2]

#### Find T score for every feature 

In [17]:
#classes
c1 = np.where(y_train=='AD ')[0]
c2 = np.where(y_train=='NL ')[0]
c3 = np.where(y_train=='SMCL ')[0]
c4 = np.where(y_train=='SQ ')[0]
c5 = np.where(y_train=='COID ')[0]

#mean of every class 
m1 = X_train[c1,:].mean(axis=0)
m2 = X_train[c2,:].mean(axis=0)
m3 = X_train[c3,:].mean(axis=0)
m4 = X_train[c4,:].mean(axis=0)
m5 = X_train[c5,:].mean(axis=0)
m = X_train.mean(axis=0)

#variance of every class 
v1 = X_train[c1,:].var(axis=0)*(c1.shape[0]-1)
v2 = X_train[c2,:].var(axis=0)*(c2.shape[0]-1)
v3 = X_train[c3,:].var(axis=0)*(c3.shape[0]-1)
v4 = X_train[c4,:].var(axis=0)*(c4.shape[0]-1)
v5 = X_train[c5,:].var(axis=0)*(c5.shape[0]-1)
s = np.sqrt((v1+v2+v3+v4+v5)/(samples-5))

t1 = np.abs(m1-m)/(np.sqrt((1/c1.shape[0])+(1/samples))*s)
t2 = np.abs(m2-m)/(np.sqrt((1/c2.shape[0])+(1/samples))*s)
t3 = np.abs(m3-m)/(np.sqrt((1/c3.shape[0])+(1/samples))*s)
t4 = np.abs(m4-m)/(np.sqrt((1/c4.shape[0])+(1/samples))*s)
t5 = np.abs(m5-m)/(np.sqrt((1/c5.shape[0])+(1/samples))*s)

tscore = np.maximum.reduce([t1,t2,t3,t4,t5])

#### Sort indexes in descending order of T score

In [18]:
sort_idxs3=(-tscore).argsort()[:tscore.shape[0]]

#### Select 33% features from T score out of selected features from f classif

In [19]:
filt_idxs3 = sort_idxs3[0:int((X_train.shape[1])*(0.333))]

#### Final filter indexes

In [20]:
X_new_train = X_train[:,filt_idxs3]
X_new_test = X_test[:,filt_idxs3]
X_new_train.shape

(162, 562)

#### KNN Classification

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_new_train, y_train)
y_pred = knn.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using KNN

In [22]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.07317073170731707
f1 score: 0.02727272727272727
confusion matrix: [[ 0 26  0  0  0]
 [ 0  3  0  0  0]
 [ 0  1  0  0  0]
 [ 0  4  0  0  0]
 [ 0  7  0  0  0]]


#### SNM Classification

In [23]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_new_train, y_train)
y_pred = clf.predict(X_new_test)

#### Accuracy, F score and Confusion matrix using SVM

In [24]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("f1 score:",metrics.f1_score(y_test, y_pred, average="macro"))
print("confusion matrix:",metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.07317073170731707
f1 score: 0.02727272727272727
confusion matrix: [[ 0 26  0  0  0]
 [ 0  3  0  0  0]
 [ 0  1  0  0  0]
 [ 0  4  0  0  0]
 [ 0  7  0  0  0]]
